In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Augmented Bonding Curve

> $price = m*supply^n$ formula. Parameterized by $supply$, $price$, and $balance$.  
> With an added entry and exit tribute

In [ ]:
#| default_exp pamm.abc

In [ ]:
#| export
import param as pm
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
from conding.pamm.bancor import BondingCurve
pn.extension()

In [ ]:
#| export
class AugmentedBondingCurve(BondingCurve):
    entry_tribute = pm.Magnitude(0.22, step=0.01)
    exit_tribute  = pm.Magnitude(0.02, step=0.01)
    total_mint_deposit = pm.Number(constant=True)
    total_mint_price = pm.Number(constant=True)
    mint_tribute  = pm.Number(constant=True)
    common_pool   = pm.Number(0, constant=True, softbounds=(0,1000), bounds=(0, None))
    
    def __init__(self, **params):
        super().__init__(**params)
        self.update_deposit()
        self.set_bounds()
        
    def set_bounds(self):
        self.param['supply'].softbounds = (self.supply/2, self.supply*1.5)
        self.param['reserve_balance'].softbounds = (self.reserve_balance/2, self.reserve_balance*1.5)
        self.param['common_pool'].softbounds = (self.common_pool/2, self.common_pool*1.5+100)
        self.param['mint_amount'].step = 1
        self.param['mint_amount'].softbounds = (-self.supply/5, self.supply/5)
        
    @pm.depends('entry_tribute', 'exit_tribute', 'deposit', watch=True)
    def update_deposit(self):
        with pm.edit_constant(self):
            if self.deposit > 0:
                self.total_mint_deposit = self.deposit / (1 - self.entry_tribute)
                self.total_mint_price = self.total_mint_deposit / self.mint_amount
                self.mint_tribute = self.total_mint_deposit - self.deposit
            elif self.deposit < 0:
                self.total_mint_deposit = self.deposit - self.deposit * self.exit_tribute
                self.total_mint_price = self.total_mint_deposit / self.mint_amount
                self.mint_tribute = self.total_mint_deposit - self.deposit
            else:
                self.total_mint_deposit = self.deposit
                self.total_mint_price = self.mint_price
                self.mint_tribute = 0
                
    def _mint(self):
        with pm.edit_constant(self):
            self.common_pool += self.mint_tribute
        super()._mint()
        
    def total_price_curve(self):
        supply = np.linspace(self.param['supply'].softbounds[0],self.param['supply'].softbounds[1]-1, num=1000)
        mint_amount = supply - self.supply
        deposit = self.get_balance_deposit(mint_amount)
        mint_total_price = deposit / (1 - self.entry_tribute) / mint_amount
        burn_total_price = (deposit - deposit * self.exit_tribute) / mint_amount
        total_price = np.where(supply > self.supply, mint_total_price, burn_total_price)
        df = pd.DataFrame({
            'Supply': supply, 
            'Total Mint Price': total_price, 
        })
        return df
    
    def view_total_price_curve(self):
        total_price_curve = self.total_price_curve()
        chart = total_price_curve.hvplot.line(
            x='Supply',
            y='Total Mint Price',
            color='brown', 
            label='Mint Price with Tribute',
            line_dash='dashed',
        )
        mint_price = (self.supply+self.mint_amount, self.total_mint_price, 'Mint Price With Tribute')
        points = pd.DataFrame([mint_price], columns=['x','y','label']).hvplot.scatter(
            x='x',
            y='y',
            by='label',
            color=['brown'],
            size=80,) 
        return chart * points
    
    @pm.depends('reserve_balance', 'supply', 'price', 'mint_amount', 'entry_tribute', 'exit_tribute')
    def view_chart(self):
        curve = self.view_price_over_supply_curve()
        points = self.view_points()
        total_price_curve = self.view_total_price_curve()
        chart = curve * total_price_curve * points
        chart.opts(
            title="Bonding Curve Math",
            legend_position="top_right",
            xlim=self.param['supply'].softbounds,
            ylim=self.param['price'].softbounds,
            width=640,
            height=640,
        )
        return chart


In [ ]:
b = AugmentedBondingCurve(supply=40)
b.view()

Row
    [0] Column(margin=5, name='AugmentedBondingCurve', width=300)
        [0] StaticText(value='<b>AugmentedBondingCurve<...)
        [1] FloatSlider(end=60.0, name='Supply', start=20.0, step=1, value=40)
        [2] FloatSlider(end=5, name='Price', start=0.01, step=0.01, value=2)
        [3] FloatSlider(end=60.0, name='Reserve balance', start=20.0, step=1, value=40)
        [4] FloatInput(disabled=True, name='Marketcap', value=80)
        [5] FloatInput(disabled=True, name='Reserve ratio', step=0.01, value=0.5)
        [6] FloatInput(disabled=True, name='N', value=1.0)
        [7] FloatInput(disabled=True, name='M', value=0.05)
        [8] FloatSlider(end=8.0, name='Mint amount', start=-8.0, step=1, value=1)
        [9] FloatInput(disabled=True, name='Deposit', value=2.0250000000000057)
        [10] FloatInput(disabled=True, name='Mint price', value=2.0250000000000057)
        [11] FloatInput(disabled=True, name='New price', value=2.0500000000000003)
        [12] Button(name='Mint')
        [13] FloatSlider(name='Entry tribute', step=0.01, value=0.22)
        [14] FloatSlider(name='Exit tribute', step=0.01, value=0.02)
        [15] FloatInput(disabled=True, name='Total mint deposit', value=2.5961538461538534)
        [16] FloatInput(disabled=True, name='Total mint price', value=2.5961538461538534)
        [17] FloatInput(disabled=True, name='Mint tribute', value=0.5711538461538477)
        [18] FloatSlider(disabled=True, end=100.0, name='Common pool')
    [1] ParamMethod(method, _pane=HoloViews)

In [ ]:
mints = np.random.randn(2) * 5 + 2
for m in mints:
    b.mint_amount = m
    b._mint()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()